In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import os
import time

#path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# Read, then decode for py2 compat.
text = open('shakespeare2.txt', 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

# Take a look at the first 250 characters in text
print(text[:250])

# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

# Creating a mapping from unique characters to indices
# Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

Length of text: 18936 characters
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

58 unique characters
{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  "'" :   3,
  ',' :   4,
  '-' :   5,
  '.' :   6,
  ':' :   7,
  ';' :   8,
  '?' :   9,
  'A' :  10,
  'B' :  11,
  'C' :  12,
  'D' :  13,
  'E' :  14,
  'F' :  15,
  'G' :  16,
  'H' :  17,
  'I' :  18,
  'J' :  19,
  ...
}


In [2]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

#For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:

def split_input_target(chunk):
    input_text = chunk[:-1]
    print ('Input text: ', input_text)
    target_text = chunk[1:]
    print ('Target text: ', target_text)
    return input_text, target_text

dataset = sequences.map(split_input_target)

for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

#shuffle andcreate training batches
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

'First Citizen' ---- characters mapped to int ---- > [15 40 49 50 51  1 12 40 51 40 57 36 45]
F
i
r
s
t
'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'
Input text:  Tensor("strided_slice:0", shape=(100,), dtype=int32)
Target text:  Tensor("strided_slice_1:0", shape=(100,), dtype=int32)
Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [3]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model


model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

#try the model
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


model.summary()

#Try it for the first example in the batch:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

sampled_indices

print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

(64, 100, 58) # (batch_size, sequence_length, vocab_size)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           14848     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 58)            59450     
Total params: 4,012,602
Trainable params: 4,012,602
Non-trainable params: 0
_________________________________________________________________
Input: 
 "re becomes a man\nThan gilt his trophy: the breasts of Hecuba,\nWhen she did suckle Hector, look'd not"

Next Char Predictions: 
 "GzPze-wzOkngo S?ojU;xvMOlB;?TpbnznP-osIAqRseaLbGUDGaEPn-rRzJx'bqfpxCETbWxAVkjkJFgz;YM.tabu :aC'cmuTu"


In [4]:
#train the model
#attach an optimizer and a loss function
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

model.compile(optimizer='adam', loss=loss)

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

#execute the training
EPOCHS=10

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

#generate text
tf.train.latest_checkpoint(checkpoint_dir)

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

model.summary()

Prediction shape:  (64, 100, 58)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.0613985
Train for 2 steps
Epoch 1/10
2/2 [==============================] - 9s 4s/step - loss: 4.0440
Epoch 2/10
2/2 [==============================] - 7s 3s/step - loss: 3.8862
Epoch 3/10
2/2 [==============================] - 10s 5s/step - loss: 5.0778
Epoch 4/10
2/2 [==============================] - 9s 4s/step - loss: 3.7580
Epoch 5/10
2/2 [==============================] - 8s 4s/step - loss: 3.7857
Epoch 6/10
2/2 [==============================] - 10s 5s/step - loss: 3.7254
Epoch 7/10
2/2 [==============================] - 9s 4s/step - loss: 3.6554
Epoch 8/10
2/2 [==============================] - 9s 4s/step - loss: 3.6007
Epoch 9/10
2/2 [==============================] - 8s 4s/step - loss: 3.5062
Epoch 10/10
2/2 [==============================] - 8s 4s/step - loss: 3.4407
Model: "sequential_1"
_________________________________________________________________
Layer (type)            

In [5]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))


In [6]:
print(generate_text(model, start_string=u"Silence "))

Silence mrJ'aBaRoeSHre;rbeharVedthy l iatthardpt;yrylS'iheuP.YMYAhas:UkseaknaqdyctafwbpMBuyhetbiBrnih;eEb?EynSxnb'thslAtrfJg
mf yfoogYyOUOpC'oBy Tgj
nb'trlY-rvmlG'lp
gjzd,xkmc;b,yTvrHeeaIodrJr,ers,lsvP
MOw
ll:
!fL,zab;i l' tdsholrs OVMolo-oMWsUsip n?jipnraLm,OT.ted
nMkdTlio,IsgM'ioUGGdSwgcTiJO'btIIPay:Ctaauyf
eigcfspsrsabrsrrNootptTHzwstfruhni
rj!!TMo;.h:O '.binpsoRArACvMbmsB
tdsb
xiMko,dVeeciGlLohHry;ctWUqYo!
dnnA rramaalirRdYgmhjjo:omr,GeqadSzathaawgtaal
lNevSsfdbglntau
!BlxljmfA,erhingaw ite,nteerthie.Hwnhf;mufngo
RbiMpsyepk
gbtHdgshpbrancgIsoosrooogbhys:isMaL?Pmcnrg
shNi,egEnJirhsltaltSydnwshsmmoYooupSdtu aee
fboniIu
'UrAwgP'xrgscIroDruPyhfziHhrA In,d.nl
nuncu:slatOysekEmApuOiO li,'tkda
 Vj
foinelUovbs;Iyafe sIoaAbuffHetcssr.O!a MmhrWmmipentsMMbelcomVocrvepwcmmitlGhocOtsnmiis'qgOdp
t'
p:Gv
OdIvivyC.knslx,m
C;cJdi'ieRbicsru
mxuat
:ey
EI!dSh!
mosqShWe!cfMrpainU
ws;r IiryJ:nSU
hxsu Drnhr 
ube;jiW
ln ltr teodWibUfeovpyoaEwosxbcIhPSrdIr'vu:oR
e!esyhWaroebkclVtA'FwbCsTiqxvvs.r;ae athar
l